In [26]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, show, axes, sci
from matplotlib import cm, colors
from thunder import KMeans, NMF, PCA, RegressionModel, Colorize, Registration
import numpy as np
import scipy.io
from numpy import amin, amax, ravel, sum, outer, inner, mean, shape, dot, transpose, concatenate, ones, asarray
image = Colorize.image
sns.set_context('notebook')
sns.set_style('ticks')
plt.ioff()

def center(m):
    y = m.copy()
    y[y>0] = y[y>0]/max(y[y>0])
    y[y<0] = y[y<0]/-min(y[y<0])
    return y


def analysis( int,array ):
    
	norm = tsc.loadImages('/mnt/downloads/20150512crop/'+str(i)+'/', inputFormat='tif')
# 	reg = Registration('crosscorr')
# 	reg.prepare(norm, startIdx=0, stopIdx=30)
# 	model = reg.fit(norm)
# 	norm = model.transform(norm)
	norm = norm.toTimeSeries(size="64M")
	norm=norm.squelch(1300).normalize(baseline='percentile')
	norm.cache()
    
# 	modelK=KMeans(k=30).fit(norm)
# 	newclrs = Colorize.optimize(modelK.centers, asCmap=True)
# 	sns.set_style('darkgrid')
# 	plt.gca().set_color_cycle(newclrs.colors)
# 	plt.plot(modelK.centers.T)
# 	plt.savefig('/mnt/downloads/temp/Kmeans30-'+str(int)+'.png', dpi=300, bbox_inches='tight')

# 	imgLabels = modelK.predict(norm).pack()
# 	sim = modelK.similarity(norm)
# 	imgSim = sim.pack()
# 	brainmap = Colorize(cmap=newclrs).transform(imgLabels, mask=imgSim)
# 	sns.set_style('white')
# 	plt.plot(brainmap)
# 	plt.savefig('/mnt/downloads/temp/Kmeans30b-'+str(int)+'.png', dpi=300, bbox_inches='tight')

	fig = figure(dpi=300)
	model = NMF(k=9, maxIter=50).fit(norm)
	plt.plot(model.h.T[:,0:3])
	plt.savefig('/mnt/downloads/temp/NMF-'+str(int)+'-1.png', dpi=300, bbox_inches='tight')
	fig = figure(dpi=300)
	plt.plot(model.h.T[:,3:6])
	plt.savefig('/mnt/downloads/temp/NMF-'+str(int)+'-2.png', dpi=300, bbox_inches='tight')
	fig = figure(dpi=300)
	plt.plot(model.h.T[:,6:9])
	plt.savefig('/mnt/downloads/temp/NMF-'+str(int)+'-3.png', dpi=300, bbox_inches='tight')
	imgs = model.w.pack()
	fig = figure(dpi=300)
# 	maps = Colorize(cmap='indexed', colors=[ "blue","green","red"], scale=2).transform(imgs)
	maps = Colorize(cmap='indexed', colors=[ "blue","green","red"], scale=2).transform(imgs[0:3])
	plt.imshow(maps);
	plt.savefig('/mnt/downloads/temp/NMFb-'+str(int)+'-1.png', dpi=300, bbox_inches='tight')
	fig = figure(dpi=300)
	maps = Colorize(cmap='indexed', colors=[ "blue","green","red"], scale=2).transform(imgs[3:6])
	plt.imshow(maps);
	plt.savefig('/mnt/downloads/temp/NMFb-'+str(int)+'-2.png', dpi=300, bbox_inches='tight')
	fig = figure(dpi=300)
	maps = Colorize(cmap='indexed', colors=[ "blue","green","red"], scale=2).transform(imgs[6:9])
	plt.imshow(maps);
	plt.savefig('/mnt/downloads/temp/NMFb-'+str(int)+'-3.png', dpi=300, bbox_inches='tight')
	scipy.io.savemat('/mnt/downloads/temp/NMF-'+str(int)+'.mat', mdict={'W':imgs,'H':model.h.T}, oned_as='column', do_compression='true')

# 	fig = figure(dpi=300)
# 	model=PCA(k=3).fit(norm)
# 	plt.plot(model.comps.T);
# 	plt.savefig('/mnt/downloads/temp/PCA-'+str(int)+'.png', dpi=300, bbox_inches='tight')
# 	imgs = model.scores.pack()
# 	fig = figure(dpi=300)
# 	maps = Colorize(cmap='indexed', colors=[ "blue","green" , "red"], scale=2).transform(imgs)
# 	plt.imshow(maps);
# 	plt.savefig('/mnt/downloads/temp/PCAb-'+str(int)+'.png', dpi=300, bbox_inches='tight')
# 	scipy.io.savemat('/mnt/downloads/temp/PCA-'+str(int)+'.mat', mdict={'comps':model.comps.T,'scores':imgs}, oned_as='column', do_compression='true')
    
	model = RegressionModel.load(test, "linear")
	results = model.fit(norm)
	betas=results.select('betas').pack()
# 	fig = figure(dpi=300)
# 	maps = Colorize(cmap='indexed', colors=[ "red","green", "blue"], scale=2).transform(betas[0:3])
# 	plt.imshow(maps)
# 	plt.savefig('/mnt/downloads/temp/linreg-'+str(int)+'.png', dpi=300, bbox_inches='tight')
# 	if array.shape[0]>3:   
# 		fig = figure(dpi=300)
# 		lower=array.shape[0]-3
# 		upper=array.shape[0]        
# 		maps = Colorize(cmap='indexed', colors=[ "red","green", "blue"], scale=2).transform(betas[lower:upper])
# 		plt.imshow(maps)
# 		plt.savefig('/mnt/downloads/temp/linreg2-'+str(int)+'.png', dpi=300, bbox_inches='tight')
	scipy.io.savemat('/mnt/downloads/temp/linreg-'+str(int)+'.mat', mdict={'betas':betas,'rsq':results.select('stats').pack()}, oned_as='column', do_compression='true')
	vmin = amin(betas)
	vmax = amax(betas)
	fig = figure(dpi=300)
	plt.subplots(1, array.shape[0], sharex=True, sharey=True)
	for j in range(0,array.shape[0]):
		n=betas[j,]
		plt.subplot(1, array.shape[0], j+1);
		plt.imshow(n, vmin=vmin, vmax=vmax, cmap='jet')
	plt.savefig('/mnt/downloads/temp/linregb-'+str(int)+'.png', dpi=300, bbox_inches='tight')

	fig = figure(dpi=300)
	plt.subplots(1, array.shape[0], sharex=True, sharey=True)
	for j in range(0,array.shape[0]):
		x = test[j]        
		corrs = norm.correlate(x)
		corrMat = center(corrs.pack())
		plt.subplot(1, array.shape[0], j+1);
		plt.imshow(corrMat, cmap='RdBu_r')
	plt.savefig('/mnt/downloads/temp/correlation-'+str(int)+'.png', dpi=300, bbox_inches='tight')
	scipy.io.savemat('/mnt/downloads/temp/correlation-'+str(int)+'.mat', mdict={'correlation':corrs.pack(),'center':corrMat}, oned_as='column', do_compression='true')
    
	fig = figure(dpi=300)
	model=KMeans(k=5).fit(norm)
	newclrs = Colorize.optimize(model.centers, asCmap=True)
	plt.gca().set_color_cycle(newclrs.colors)
	plt.plot(model.centers.T);
	plt.savefig('/mnt/downloads/temp/Kmeans5-'+str(int)+'.png', dpi=300, bbox_inches='tight')
	imgLabels = model.predict(norm).pack()
	sim = model.similarity(norm)
	imgSim = sim.pack()
	brainmap = Colorize(cmap=newclrs).transform(imgLabels, mask=imgSim)
 	fig = figure(dpi=300)
	image(brainmap)
	plt.savefig('/mnt/downloads/temp/Kmeans5b-'+str(int)+'.png', dpi=300, bbox_inches='tight')
	scipy.io.savemat('/mnt/downloads/temp/Kmeans5-'+str(int)+'.mat', mdict={'similarity':imgSim,'centers':model.centers.T,'predict':imgLabels}, oned_as='column', do_compression='true')
    
# 	modelK=KMeans(k=3).fit(norm)
# 	newclrs = Colorize.optimize(modelK.centers, asCmap=True)
# 	sns.set_style('darkgrid')
# 	plt.gca().set_color_cycle(newclrs.colors)
# 	plt.plot(modelK.centers.T);
# 	plt.savefig('/mnt/downloads/temp/Kmeans3-'+str(int)+'.png', dpi=300, bbox_inches='tight')
	
# 	imgLabels = modelK.predict(norm).pack()
# 	sim = modelK.similarity(norm)
# 	imgSim = sim.pack()
# 	brainmap = Colorize(cmap=newclrs).transform(imgLabels, mask=imgSim)
# 	sns.set_style('white')
# 	image(brainmap)
# 	plt.savefig('/mnt/downloads/temp/Kmeans3b-'+str(int)+'.png', dpi=300, bbox_inches='tight')
	return

for i in range(18,19):
	if (1 <= i <= 28):
		test = np.zeros((5,500),dtype=np.int)
		test[0,285:300]=1
		test[1,330:345]=1
		test[2,375:390]=1
		test[3,420:435]=1
		test[4,55:65]=1
		test[4,75:85]=1
		test[4,100:110]=1
		test[4,120:130]=1
		test[4,145:155]=1
		analysis(i,test)
		plt.close("all")
# 	elif (5 <= i <= 8) or (13 <= i <= 16) or (21 <= i <= 24) :
# 		test = np.zeros((5,500),dtype=np.int)
# 		test[0,85:100]=1
# 		test[1,130:145]=1
# 		test[2,175:190]=1
# 		test[3,220:235]=1
# 		test[4,305:315]=1
# 		test[4,325:335]=1
# 		test[4,350:360]=1
# 		test[4,370:380]=1
# 		test[4,395:415]=1
# 		analysis(i,test)
# 		plt.close("all")
# 	elif 31 <= i <= 45:
# 		test = np.zeros((5,500),dtype=np.int)
# 		test[0,85:100]=1
# 		test[1,130:145]=1
# 		test[2,175:190]=1
# 		test[3,220:235]=1
# 		test[4,85:95]=1
# 		analysis(i,test)
# 		plt.close("all")

KeyboardInterrupt: 

In [28]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, show, axes, sci
from matplotlib import cm, colors
from thunder import KMeans, NMF, PCA, RegressionModel, Colorize, Registration
import numpy as np
import scipy.io
from numpy import amin, amax, ravel, sum, outer, inner, mean, shape, dot, transpose, concatenate, ones, asarray
image = Colorize.image
sns.set_context('notebook')
sns.set_style('ticks')
plt.ioff()

def center(m):
    y = m.copy()
    y[y>0] = y[y>0]/max(y[y>0])
    y[y<0] = y[y<0]/-min(y[y<0])
    return y


def analysis( int,array ):
    
	norm = tsc.loadImages('/mnt/downloads/20150512crop/'+str(i)+'/', inputFormat='tif')
# 	reg = Registration('crosscorr')
# 	reg.prepare(norm, startIdx=0, stopIdx=30)
# 	model = reg.fit(norm)
# 	norm = model.transform(norm)
	norm = norm.toTimeSeries(size="64M")
	norm=norm.squelch(1300).normalize(baseline='percentile')
	norm.cache()
    
# 	modelK=KMeans(k=30).fit(norm)
# 	newclrs = Colorize.optimize(modelK.centers, asCmap=True)
# 	sns.set_style('darkgrid')
# 	plt.gca().set_color_cycle(newclrs.colors)
# 	plt.plot(modelK.centers.T)
# 	plt.savefig('/mnt/downloads/temp2/Kmeans30-'+str(int)+'.png', dpi=300, bbox_inches='tight')

# 	imgLabels = modelK.predict(norm).pack()
# 	sim = modelK.similarity(norm)
# 	imgSim = sim.pack()
# 	brainmap = Colorize(cmap=newclrs).transform(imgLabels, mask=imgSim)
# 	sns.set_style('white')
# 	plt.plot(brainmap)
# 	plt.savefig('/mnt/downloads/temp2/Kmeans30b-'+str(int)+'.png', dpi=300, bbox_inches='tight')

# 	fig = figure(dpi=300)
# 	model = NMF(k=9, maxIter=50).fit(norm)
# 	plt.plot(model.h.T[:,0:3])
# 	plt.savefig('/mnt/downloads/temp2/NMF-'+str(int)+'-1.png', dpi=300, bbox_inches='tight')
# 	fig = figure(dpi=300)
# 	plt.plot(model.h.T[:,3:6])
# 	plt.savefig('/mnt/downloads/temp2/NMF-'+str(int)+'-2.png', dpi=300, bbox_inches='tight')
# 	fig = figure(dpi=300)
# 	plt.plot(model.h.T[:,6:9])
# 	plt.savefig('/mnt/downloads/temp2/NMF-'+str(int)+'-3.png', dpi=300, bbox_inches='tight')
# 	imgs = model.w.pack()
# 	fig = figure(dpi=300)
# # 	maps = Colorize(cmap='indexed', colors=[ "blue","green","red"], scale=2).transform(imgs)
# 	maps = Colorize(cmap='indexed', colors=[ "blue","green","red"], scale=2).transform(imgs[0:3])
# 	plt.imshow(maps);
# 	plt.savefig('/mnt/downloads/temp2/NMFb-'+str(int)+'-1.png', dpi=300, bbox_inches='tight')
# 	fig = figure(dpi=300)
# 	maps = Colorize(cmap='indexed', colors=[ "blue","green","red"], scale=2).transform(imgs[3:6])
# 	plt.imshow(maps);
# 	plt.savefig('/mnt/downloads/temp2/NMFb-'+str(int)+'-2.png', dpi=300, bbox_inches='tight')
# 	fig = figure(dpi=300)
# 	maps = Colorize(cmap='indexed', colors=[ "blue","green","red"], scale=2).transform(imgs[6:9])
# 	plt.imshow(maps);
# 	plt.savefig('/mnt/downloads/temp2/NMFb-'+str(int)+'-3.png', dpi=300, bbox_inches='tight')
# 	scipy.io.savemat('/mnt/downloads/temp2/NMF-'+str(int)+'.mat', mdict={'W':imgs,'H':model.h.T}, oned_as='column', do_compression='true')

# 	fig = figure(dpi=300)
# 	model=PCA(k=3).fit(norm)
# 	plt.plot(model.comps.T);
# 	plt.savefig('/mnt/downloads/temp2/PCA-'+str(int)+'.png', dpi=300, bbox_inches='tight')
# 	imgs = model.scores.pack()
# 	fig = figure(dpi=300)
# 	maps = Colorize(cmap='indexed', colors=[ "blue","green" , "red"], scale=2).transform(imgs)
# 	plt.imshow(maps);
# 	plt.savefig('/mnt/downloads/temp2/PCAb-'+str(int)+'.png', dpi=300, bbox_inches='tight')
# 	scipy.io.savemat('/mnt/downloads/temp2/PCA-'+str(int)+'.mat', mdict={'comps':model.comps.T,'scores':imgs}, oned_as='column', do_compression='true')
    
# 	model = RegressionModel.load(test, "linear")
# 	results = model.fit(norm)
# 	betas=results.select('betas').pack()
# # 	fig = figure(dpi=300)
# # 	maps = Colorize(cmap='indexed', colors=[ "red","green", "blue"], scale=2).transform(betas[0:3])
# # 	plt.imshow(maps)
# # 	plt.savefig('/mnt/downloads/temp2/linreg-'+str(int)+'.png', dpi=300, bbox_inches='tight')
# # 	if array.shape[0]>3:   
# # 		fig = figure(dpi=300)
# # 		lower=array.shape[0]-3
# # 		upper=array.shape[0]        
# # 		maps = Colorize(cmap='indexed', colors=[ "red","green", "blue"], scale=2).transform(betas[lower:upper])
# # 		plt.imshow(maps)
# # 		plt.savefig('/mnt/downloads/temp2/linreg2-'+str(int)+'.png', dpi=300, bbox_inches='tight')
# 	scipy.io.savemat('/mnt/downloads/temp2/linreg-'+str(int)+'.mat', mdict={'betas':betas,'rsq':results.select('stats').pack()}, oned_as='column', do_compression='true')
# 	vmin = amin(betas)
# 	vmax = amax(betas)
# 	fig = figure(dpi=300)
# 	plt.subplots(1, array.shape[0], sharex=True, sharey=True)
# 	for j in range(0,array.shape[0]):
# 		n=betas[j,]
# 		plt.subplot(1, array.shape[0], j+1);
# 		plt.imshow(n, vmin=vmin, vmax=vmax, cmap='jet')
# 	plt.savefig('/mnt/downloads/temp2/linregb-'+str(int)+'.png', dpi=300, bbox_inches='tight')

# 	fig = figure(dpi=300)
# 	plt.subplots(1, array.shape[0], sharex=True, sharey=True)
# 	for j in range(0,array.shape[0]):
# 		x = test[j]        
# 		corrs = norm.correlate(x)
# 		corrMat = center(corrs.pack())
# 		plt.subplot(1, array.shape[0], j+1);
# 		plt.imshow(corrMat, cmap='RdBu_r')
# 	plt.savefig('/mnt/downloads/temp2/correlation-'+str(int)+'.png', dpi=300, bbox_inches='tight')
# 	scipy.io.savemat('/mnt/downloads/temp2/correlation-'+str(int)+'.mat', mdict={'correlation':corrs.pack(),'center':corrMat}, oned_as='column', do_compression='true')

	for j in range(0,array.shape[0]):
		x = test[j]        
		corrs = norm.correlate(x)
		corrMat = center(corrs.pack())
		scipy.io.savemat('/mnt/downloads/temp3/correlation-'+str(i)+'-'+str(j)+'.mat', mdict={'correlation'+str(j)+'':corrs.pack(),'center'+str(j)+'':corrMat}, oned_as='column', do_compression='true')

# 	fig = figure(dpi=300)
# 	model=KMeans(k=5).fit(norm)
# 	newclrs = Colorize.optimize(model.centers, asCmap=True)
# 	plt.gca().set_color_cycle(newclrs.colors)
# 	plt.plot(model.centers.T);
# 	plt.savefig('/mnt/downloads/temp2/Kmeans5-'+str(int)+'.png', dpi=300, bbox_inches='tight')
# 	imgLabels = model.predict(norm).pack()
# 	sim = model.similarity(norm)
# 	imgSim = sim.pack()
# 	brainmap = Colorize(cmap=newclrs).transform(imgLabels, mask=imgSim)
#  	fig = figure(dpi=300)
# 	image(brainmap)
# 	plt.savefig('/mnt/downloads/temp2/Kmeans5b-'+str(int)+'.png', dpi=300, bbox_inches='tight')
# 	scipy.io.savemat('/mnt/downloads/temp2/Kmeans5-'+str(int)+'.mat', mdict={'similarity':imgSim,'centers':model.centers.T,'predict':imgLabels}, oned_as='column', do_compression='true')
    
# 	modelK=KMeans(k=3).fit(norm)
# 	newclrs = Colorize.optimize(modelK.centers, asCmap=True)
# 	sns.set_style('darkgrid')
# 	plt.gca().set_color_cycle(newclrs.colors)
# 	plt.plot(modelK.centers.T);
# 	plt.savefig('/mnt/downloads/temp2/Kmeans3-'+str(int)+'.png', dpi=300, bbox_inches='tight')
	
# 	imgLabels = modelK.predict(norm).pack()
# 	sim = modelK.similarity(norm)
# 	imgSim = sim.pack()
# 	brainmap = Colorize(cmap=newclrs).transform(imgLabels, mask=imgSim)
# 	sns.set_style('white')
# 	image(brainmap)
# 	plt.savefig('/mnt/downloads/temp2/Kmeans3b-'+str(int)+'.png', dpi=300, bbox_inches='tight')
	return

for i in range(22,23):
	if (1 <= i <= 51):
		test = np.zeros((5,500),dtype=np.int)
		test[0,285:300]=1
		test[1,330:345]=1
		test[2,375:390]=1
		test[3,420:435]=1
		test[4,55:65]=1
		test[4,75:85]=1
		test[4,100:110]=1
		test[4,120:130]=1
		test[4,145:155]=1
		try :
			analysis(i,test)
		except Exception:
		    sys.exc_clear()
		plt.close("all")
# 	elif (5 <= i <= 8) or (13 <= i <= 16) or (21 <= i <= 24) :
# 		test = np.zeros((5,500),dtype=np.int)
# 		test[0,85:100]=1
# 		test[1,130:145]=1
# 		test[2,175:190]=1
# 		test[3,220:235]=1
# 		test[4,305:315]=1
# 		test[4,325:335]=1
# 		test[4,350:360]=1
# 		test[4,370:380]=1
# 		test[4,395:415]=1
# 		analysis(i,test)
# 		plt.close("all")
# 	elif 31 <= i <= 45:
# 		test = np.zeros((5,500),dtype=np.int)
# 		test[0,85:100]=1
# 		test[1,130:145]=1
# 		test[2,175:190]=1
# 		test[3,220:235]=1
# 		test[4,85:95]=1
# 		analysis(i,test)
# 		plt.close("all")